
# Aharonov–Bohm Sensor (Arman & Chase) — Algorithmic Model

This notebook simulates the **core signal chain** described in US 8,389,948 B2 (*"Aharonov–Bohm Sensor"* by Arman & Chase).
It models an **electron interferometer** whose output intensity depends on the **Aharonov–Bohm phase** induced by **enclosed magnetic flux** `Φ(t)` from a toroidal source (ideally with **B ≈ 0** outside).

**What you can do here**
- Define **toroid** geometry → compute flux vs current `Φ = κ I`, with `κ` a geometry-dependent constant.
- Drive the toroid with **DC/AC/modulated currents**; add **noise & drift**.
- Compute **AB phase** `Δφ(t) = (e/ħ) Φ(t)` and interferometer **intensity** `I(t) = I0[1 + V cos(Δφ + φ0)]`.
- Add **classical leakage** terms (tiny B-field leakage, vibration, photocurrent noise) to test discrimination.
- Run **lock-in demodulation** at the drive frequency to recover phase under noise.
- Estimate sensitivity; sweep parameters (visibility, coherence length, leakage, bandwidth).
- Fit unknown parameters from synthetic data using simple least-squares / maximum-likelihood.
  
> **Note:** This is a *systems-level* engineering model for exploration. It is not a full quantum transport or electron-optics simulator.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, Dict
from numpy.random import default_rng

rng = default_rng(42)

# Physical constants
hbar = 1.054_571_817e-34  # J·s
e_charge = 1.602_176_634e-19  # C
mu0 = 4*np.pi*1e-7  # H/m

def show(ax, title):
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('Time (s)')


In [ ]:

@dataclass
class Toroid:
    N: int         # turns
    mu_r: float    # relative permeability of core
    r_mean: float  # mean radius (m)
    A_core: float  # core cross-sectional area (m^2)
    
    def kappa(self) -> float:
        """Flux constant Φ = κ I (ideal toroid). B = μ0 μr N I / (2π r_mean); Φ = B * A_core."""
        return (mu0 * self.mu_r * self.N * self.A_core) / (2*np.pi*self.r_mean)

@dataclass
class Interferometer:
    I0: float      # average intensity (arb)
    V: float       # fringe visibility [0..1]
    phi0: float    # static bias phase (rad)
    Lc: float      # coherence length (m)
    dL: float      # arm length mismatch (m)
    
    def visibility_eff(self) -> float:
        """Visibility reduction due to finite coherence length (Gaussian model)."""
        if self.Lc <= 0:
            return 0.0
        return self.V * np.exp(- (self.dL/self.Lc)**2 )
    
    def intensity(self, dphi: np.ndarray) -> np.ndarray:
        Veff = self.visibility_eff()
        return self.I0 * (1.0 + Veff * np.cos(dphi + self.phi0))


In [ ]:

def drive_current(t, DC=0.0, AC_amp=0.0, f0=1_237.0, pm=None):
    """Toroid current: DC + AC tone + optional phase modulation (pm dict: {'beta':..., 'fpm':...})."""
    carrier = AC_amp * np.cos(2*np.pi*f0*t)
    if pm:
        beta = pm.get('beta', 0.1)
        fpm = pm.get('fpm', f0/10)
        carrier = AC_amp * np.cos(2*np.pi*f0*t + beta*np.sin(2*np.pi*fpm*t))
    return DC + carrier

def add_noise(sig, snr_db=20.0):
    p_sig = np.mean(sig**2) + 1e-30
    p_noise = p_sig / (10**(snr_db/10))
    noise = rng.normal(0, np.sqrt(p_noise), size=sig.shape)
    return sig + noise

def add_leakage_terms(t, leak_B_amp=0.0, f0=1_237.0, vib_amp=0.0, vib_f=200.0):
    """Classical contaminations: tiny B leakage and vibration (intensity offset modulation)."""
    leak = leak_B_amp * np.cos(2*np.pi*f0*t)         # synchronous with drive
    vib  = vib_amp * np.sin(2*np.pi*vib_f*t)         # unrelated mechanical
    return leak, vib

def lock_in(sig, t, fref, tau=1e-3):
    """IQ lock-in demodulation at fref with single-pole LPF (time-constant tau)."""
    dt = np.mean(np.diff(t))
    x = sig * np.cos(2*np.pi*fref*t)
    y = sig * np.sin(2*np.pi*fref*t)
    # single-pole IIR low-pass
    alpha = dt / (tau + dt)
    xi, yi = 0.0, 0.0
    X, Y = np.zeros_like(sig), np.zeros_like(sig)
    for i,(xa,ya) in enumerate(zip(x,y)):
        xi = (1-alpha)*xi + alpha*xa
        yi = (1-alpha)*yi + alpha*ya
        X[i] = xi; Y[i] = yi
    return X, Y


In [ ]:

# --- Parameters ---
T = 0.1                 # total time (s)
fs = 200_000            # sample rate (Hz)
t = np.arange(0, T, 1/fs)

# Toroid (example): medium-µ core, modest geometry
tor = Toroid(N=400, mu_r=2000.0, r_mean=0.03, A_core=3e-5)  # r=3 cm, area=3e-5 m^2
kappa = tor.kappa()     # Φ = κ I

# Interferometer
itf = Interferometer(I0=1.0, V=0.9, phi0=np.deg2rad(20), Lc=0.05, dL=0.005)

# Drive
f0 = 1_237.0
I = drive_current(t, DC=0.0, AC_amp=0.2, f0=f0, pm={'beta':0.4, 'fpm':123.7})

# Flux, AB phase
Phi = kappa * I                     # Weber
dphi = (e_charge/hbar) * Phi        # radians

# Ideal interferometer intensity
I_det = itf.intensity(dphi)

# Add classical leakages & noise
leak, vib = add_leakage_terms(t, leak_B_amp=0.02, f0=f0, vib_amp=0.01, vib_f=200.0)
I_meas = add_noise(I_det + leak + vib, snr_db=25)

# Lock-in at f0
X, Y = lock_in(I_meas - np.mean(I_meas), t, fref=f0, tau=1e-3)
R = np.sqrt(X**2 + Y**2)
phi_lock = np.unwrap(np.angle(X + 1j*Y))

# Plots
fig, ax = plt.subplots(3, 1, figsize=(8, 8))
ax[0].plot(t[:4000], I_meas[:4000])
ax[0].set_title('Measured detector signal (snippet)'); ax[0].grid(True, alpha=0.3); ax[0].set_xlabel('Time (s)')
ax[1].plot(t, X, label='I (in-phase)'); ax[1].plot(t, Y, label='Q (quadrature)')
ax[1].legend(); ax[1].set_title('Lock-in I/Q (τ=1 ms)'); ax[1].grid(True, alpha=0.3); ax[1].set_xlabel('Time (s)')
ax[2].plot(t, R); ax[2].set_title('Lock-in magnitude R'); ax[2].grid(True, alpha=0.3); ax[2].set_xlabel('Time (s)')
plt.tight_layout(); plt.show()


In [ ]:

# Sensitivity vs fringe visibility
def simulate_visibility_sweep(Vvals, itf_template, dphi, t, f0):
    amps = []
    for V in Vvals:
        itf2 = Interferometer(I0=itf_template.I0, V=V, phi0=itf_template.phi0,
                              Lc=itf_template.Lc, dL=itf_template.dL)
        I_det2 = itf2.intensity(dphi)
        I_meas2 = add_noise(I_det2, snr_db=25)
        X2, Y2 = lock_in(I_meas2 - np.mean(I_meas2), t, fref=f0, tau=1e-3)
        amps.append(np.mean(np.sqrt(X2**2 + Y2**2)[int(0.5*len(t)):]))  # steady-state
    return np.array(amps)

Vvals = np.linspace(0.1, 0.95, 10)
amps = simulate_visibility_sweep(Vvals, itf, dphi, t, f0)

fig, ax = plt.subplots(figsize=(7,4))
ax.plot(Vvals, amps, marker='o')
ax.set_xlabel('Fringe visibility V'); ax.set_ylabel('Lock-in amplitude (arb)')
ax.set_title('Sensitivity vs interferometer visibility'); ax.grid(True, alpha=0.3)
plt.show()


In [ ]:

# Leakage discrimination demo
def leakage_scan(leak_levels, I_det, t, f0):
    Rlevels = []
    for leak_amp in leak_levels:
        I_measL = add_noise(I_det + leak_amp*np.cos(2*np.pi*f0*t), snr_db=25)
        Xl, Yl = lock_in(I_measL - np.mean(I_measL), t, fref=f0, tau=1e-3)
        Rlevels.append(np.mean(np.sqrt(Xl**2 + Yl**2)[int(0.5*len(t)):]))  # after settling
    return np.array(Rlevels)

leak_levels = np.linspace(0, 0.05, 8)
Rleak = leakage_scan(leak_levels, I_det, t, f0)

fig, ax = plt.subplots(figsize=(7,4))
ax.plot(leak_levels, Rleak, marker='s')
ax.set_xlabel('Injected synchronous leakage amplitude (arb)')
ax.set_ylabel('Lock-in R (arb)')
ax.set_title('Effect of synchronous classical leakage on recovered amplitude'); ax.grid(True, alpha=0.3)
plt.show()


In [ ]:

# Parameter estimation via least squares
from scipy.optimize import least_squares

def synth_measure(t, f0, kappa, Iamp=0.2, V=0.9, phi0=np.deg2rad(20), Lc=0.05, dL=0.005, leak_amp=0.0, snr_db=25):
    I = drive_current(t, DC=0.0, AC_amp=Iamp, f0=f0)
    Phi = kappa * I
    dphi = (e_charge/hbar) * Phi
    itf_tmp = Interferometer(I0=1.0, V=V, phi0=phi0, Lc=Lc, dL=dL)
    I_det_tmp = itf_tmp.intensity(dphi)
    meas = add_noise(I_det_tmp + leak_amp*np.cos(2*np.pi*f0*t), snr_db=snr_db)
    return meas

y_obs = synth_measure(t, f0, kappa, Iamp=0.18, V=0.8, leak_amp=0.01, snr_db=22)

def residual(theta):
    Iamp, V, leak = theta
    y_hat = synth_measure(t, f0, kappa, Iamp=Iamp, V=V, leak_amp=leak, snr_db=1e9)  # model without noise
    return y_hat - y_obs

theta0 = np.array([0.15, 0.6, 0.0])
bounds = ([0.0, 0.0, 0.0],[1.0, 1.0, 0.2])
res = least_squares(residual, theta0, bounds=bounds, max_nfev=50)

Iamp_est, V_est, leak_est = res.x
y_fit = synth_measure(t, f0, kappa, Iamp=Iamp_est, V=V_est, leak_amp=leak_est, snr_db=1e9)

fig, ax = plt.subplots(2,1, figsize=(8,6))
idx = slice(0, 5000)
ax[0].plot(t[idx], y_obs[idx], label='Observed')
ax[0].plot(t[idx], y_fit[idx], label='Fit', linestyle='--')
ax[0].legend(); ax[0].set_title('Data vs fitted model (snippet)'); ax[0].grid(True, alpha=0.3); ax[0].set_xlabel('Time (s)')
ax[1].plot(t, y_obs - y_fit); ax[1].set_title('Residual'); ax[1].grid(True, alpha=0.3); ax[1].set_xlabel('Time (s)')
plt.tight_layout(); plt.show()

print(f"Estimated Iamp={Iamp_est:.3f}, V={V_est:.3f}, leak={leak_est:.3f}")
